In [ ]:
!pip install datasets sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=1902e7ef6176619c0e3c873f4cddd23e4e2da41c0

In [ ]:
from datasets import load_dataset

dataset = load_dataset('ms_marco', 'v1.1')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

Dataset ms_marco downloaded and prepared to /root/.cache/huggingface/datasets/ms_marco/v1.1/1.1.0/b6a62715fa5219aea5275dd3556601004cd63945cb63e36e022f77bb3cbbca84. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
subset = dataset['test']

In [ ]:
queries_infos = []
queries = []
corpus = []

for sample in subset:
    query_type = sample['query_type']
    if query_type != 'entity':
        continue
    query_id = sample['query_id']
    query_str = sample['query']
    passages_dict = sample['passages']
    is_selected_lst = passages_dict['is_selected']
    passage_text_lst = passages_dict['passage_text']
    query_info = {
        'query_id': query_id,
        'query': query_str,
        'relevant_docs': []
    }
    current_len_corpus = len(corpus)
    for idx in range(len(is_selected_lst)):
        if is_selected_lst[idx] == 1:
            doc_idx = current_len_corpus + idx
            query_info['relevant_docs'].append(doc_idx)

    if query_info['relevant_docs'] == []:
        continue

    queries.append(query_str)
    queries_infos.append(query_info)
    corpus += passage_text_lst

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

## **Custom search function**

In [ ]:
from sentence_transformers import util

def similarity(query_embeddings, corpus_embeddings):
    return util.cos_sim(query_embeddings, corpus_embeddings)[0]

In [ ]:
def ranking(query, top_k=10):
    query_embeddings = model.encode(
        query,
        convert_to_tensor=True
    )
    cos_scores = similarity(
        query_embeddings,
        corpus_embeddings
    )
    top_results = torch.topk(cos_scores, k=top_k)

    return top_results

In [ ]:
custom_queries = ['what is the official language in Fiji']

top_k = min(5, len(corpus))
for query in custom_queries:
    top_results = ranking(query, top_k)

    print("Query:", query)
    print("\n======================")
    print(f"Top {top_k} most similar sentences in corpus:\n")

    for idx, (score, doc_idx) in enumerate(
        zip(top_results[0], top_results[1])
    ):
        print(f'Document rank {idx + 1}:')
        print(corpus[doc_idx], f'\n(Score: {score:.4f})', '\n')

Query: what is the official language in Fiji

Top 5 most similar sentences in corpus:

Document rank 1:
The official languages. Fiji’s 1997 Constitution established Fijian as one of the official languages of the country. Fijian is an Austronesian language, a grouping that includes thousands of other languages spanning the globe. The language is of the Malayo-Polynesian family, not too different from Hawaiian and Maori. 
(Score: 0.8663) 

Document rank 2:
Fiji has three official languages under the 1997 constitution (and not revoked by the 2013 Constitution): English, Fijian and Hindi. Fijian is spoken either as a first or second language by indigenous Fijians who make up around 54% of the population. 
(Score: 0.8464) 

Document rank 3:
The Republic of the Fiji Islands citizens speak British English. Fijian and Fiji-Hindi is the second language. Other major language that is taught in elementary/primary schools and high schools are Urdu and French. Urdu and French is never considered to 